In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The Data

In [3]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df = pd.read_csv('../../data/more_feats_0.2.csv')
#df= pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df =pd.read_csv('../../data/more_features/more_feats_new_labels_0.1_noisy_0.2.csv')
#df = pd.read_csv('../../data/more_features/more_feats_0.3.csv')
#df = utils.balance_dataset(df, 8000)
#df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,12.190830,163.097819,3.084127,5.923027,232.339305,79.550470,-1.000000,3.748689,-1.000000,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Anemia of chronic disease
1,9.944819,187.086684,5.096256,-1.000000,-1.000000,100.121185,-1.000000,5.857746,72.837294,0,1.146823,-1.000000,-1.000000,-1.000000,-1.000000,20.943783,-1.000000,-1.000000,Unspecified anemia
2,13.703345,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,80.372015,5.684361,45.702318,0,1.789854,206.817706,1.551467,90.217875,76.009442,22.651663,12.551513,-1.000000,No anemia
3,7.346123,27.669632,-1.000000,-1.000000,428.089083,77.375356,95.912445,6.469686,52.594562,0,0.222080,297.319109,1.265341,126.999491,46.998397,77.183582,8.518821,119.113878,Anemia of chronic disease
4,12.295548,225.097199,3.827717,0.000000,-1.000000,102.137453,-1.000000,-1.000000,-1.000000,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,Unspecified anemia


In [4]:
df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
age                      0
gender                   0
indirect_bilirubin       0
transferrin              0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
label                    0
dtype: int64

In [5]:
utils.get_dt_performance(df)

(0.9618571428571429,
 0.9523921902607798,
 0.9733744195819131,
 datetime.timedelta(microseconds=3968))

In [6]:
df.label.value_counts()

No anemia                               10000
Anemia of chronic disease                9756
Iron deficiency anemia                   9267
Unspecified anemia                       9033
Aplastic anemia                          9020
Vitamin B12/Folate deficiency anemia     9000
Hemolytic anemia                         8976
Inconclusive diagnosis                   4948
Name: label, dtype: int64

In [7]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 18), (14000, 18), (56000,), (14000,))

In [8]:
X_test[0]

array([ 10.34995981,  -1.        ,   0.99855381,  -1.        ,
        -1.        ,  86.411147  , 111.70748795,   4.94433984,
        27.50298959,   0.        ,   0.27929003, 157.7912435 ,
         0.72896824,  19.37271239,  51.43928931,  43.59894376,
         7.55852452,  64.72339742])

In [9]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'age',
 'gender',
 'indirect_bilirubin',
 'transferrin',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose']

In [10]:
len(action_list)

26

In [11]:
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,12.190830,163.097819,3.084127,5.923027,232.339305,79.550470,-1.000000,3.748689,-1.000000,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3
1,9.944819,187.086684,5.096256,-1.000000,-1.000000,100.121185,-1.000000,5.857746,72.837294,0,1.146823,-1.000000,-1.000000,-1.000000,-1.000000,20.943783,-1.000000,-1.000000,2
2,13.703345,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,80.372015,5.684361,45.702318,0,1.789854,206.817706,1.551467,90.217875,76.009442,22.651663,12.551513,-1.000000,0
3,7.346123,27.669632,-1.000000,-1.000000,428.089083,77.375356,95.912445,6.469686,52.594562,0,0.222080,297.319109,1.265341,126.999491,46.998397,77.183582,8.518821,119.113878,3
4,12.295548,225.097199,3.827717,0.000000,-1.000000,102.137453,-1.000000,-1.000000,-1.000000,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2


#### Training 

In [ ]:
# %%time
# timesteps = int(2e6)
# dqn_model = utils.stable_dqn3(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn3_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
#for steps in [int(6e6), int(6.5e6), int(7e6), int(7.5e6), int(8e6), int(8.5e6), int(9e6)]:
for steps in [int(11e6), int(12e6)]:
    #start_time = datetime.now()
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, 
                                  f'../../models/many_features/0.1/dqn3_by_type_new_labels_noisy_2_{steps}')
    #end_time = datetime.now()
    #print(f'The duration for {steps} steps is {end_time-start_time}')

#### Testing

In [12]:
training_env = utils.create_env(X_train, y_train)
dqn_model = utils.load_dqn3('../../models/many_features/0.1/dqn3_by_type_new_labels_noisy_2_9000000', training_env)
test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
test_df.head()

Using stable baselines 3
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Count: 2800
Count: 5600
Count: 8400
Count: 11200
Count: 14000
Testing done.....


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,4.0,0.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Aplastic anemia]",6.0,6.0
1,4.0,1.0,1.0,1.0,0.0,"[hemoglobin, mcv, segmented_neutrophils, Unspe...",2.0,2.0
2,4.0,2.0,1.0,1.0,0.0,"[hemoglobin, mcv, segmented_neutrophils, Unspe...",2.0,2.0
3,4.0,3.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Hemolytic anemia]",5.0,5.0
4,4.0,4.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Hemolytic anemia]",5.0,5.0


In [13]:
test_df.y_pred.value_counts()

7.0    2158
0.0    2052
3.0    2035
6.0    1814
1.0    1520
4.0    1510
5.0    1465
2.0    1446
Name: y_pred, dtype: int64

In [14]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

83.26428571428572

In [15]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(4.1275, 0.5271428571428571)

In [16]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.8326428571428571, 0.82752211440918, 0.9076263112228821)

In [17]:
test_df.y_pred.unique()

array([6., 2., 5., 0., 4., 3., 7., 1.])

In [18]:
test_df[test_df.y_pred==4]

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
8,6.0,8.0,1.0,1.0,0.0,"[hemoglobin, mcv, gender, tibc, ferritin, Iron...",4.0,4.0
16,5.0,16.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, ferritin, Iron deficie...",4.0,4.0
23,5.0,23.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, tibc, Iron deficienc...",4.0,4.0
31,4.0,31.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, Iron deficiency anemia]",4.0,4.0
41,5.0,41.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, tibc, Iron deficienc...",4.0,4.0
...,...,...,...,...,...,...,...,...
13965,6.0,13965.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, tibc, ferritin, Iron...",4.0,4.0
13968,6.0,13968.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, tibc, ferritin, Iron...",4.0,4.0
13989,5.0,13989.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, ferritin, Iron deficie...",4.0,4.0
13996,5.0,13996.0,1.0,1.0,0.0,"[hemoglobin, mcv, tibc, ferritin, Iron deficie...",4.0,4.0


#### Saving files

In [19]:
test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_2_9000000.csv', index=False)
success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_2_9000000.csv', index=False)

#### Confusion matrix and classification report

In [ ]:
test_df = pd.read_csv('../../test_dfs/many_features/0.1/test_df3_6500000.csv')
test_df.head()

In [ ]:
utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# def plot_confusion_matrix(y_actual, y_pred, save=False, filename=False):
#     from sklearn.metrics import confusion_matrix
#     cm = confusion_matrix(y_actual, y_pred)
#     cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5, 6], columns = [0, 1, 2, 3, 4, 5, 6], dtype='object')
#     #cm_df = pd.DataFrame(cm, index = constants.CLASS_DICT.keys(), columns = constants.CLASS_DICT.keys())
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_df, annot=True)
#     plt.title('Confusion Matrix')
#     plt.ylabel('Actual Anemia')
#     plt.xlabel('Predicted Anemia')
#     plt.tight_layout()
#     if save:
#         plt.savefig(filename)
#     plt.show()
#     plt.close()

In [ ]:
utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])